# CIFAR Experiments

Notebook for evaluating our models on the classic CIFAR dataset.

## Setup

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T

import time
import numpy as np
import matplotlib.pyplot as plt

from data_utils import get_CIFAR10_data

In [5]:
%load_ext autoreload
%autoreload 2

from modulesStackable import OctConv2dStackable, get_stacked_4, get_SixLayerConvNet
from octconv_tests import test_octconv_shapes, test_octconv_as_conv


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
#231N Assignment 2

USE_GPU = True
print_every = 100

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


## Load CIFAR Data

The following code should download the CIFAR data automatically. If not, you can do so by navigating to the `datasets` directory and running the `$ ./get_datasets.sh` command from the command line.

In [13]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Train Vanilla Model on CIFAR

In [14]:
def check_accuracy(loader, model):
    """
    Checks accuracy of model on given data loader.
    Code from 231N Assignment 2.
    """
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [17]:
def train_cifar(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    Code from 231N assignment 2.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()

In [ ]:
vanilla_model = get_SixLayerConvNet()

learning_rate = 5e-4
optimizer = optim.Adam(vanilla_model.parameters(), lr=learning_rate)

train_cifar(vanilla_model, optimizer, epochs=10)

Iteration 0, loss = 2.8215
Checking accuracy on validation set
Got 110 / 1000 correct (11.00)

Iteration 100, loss = 1.9686
Checking accuracy on validation set
Got 294 / 1000 correct (29.40)

Iteration 200, loss = 1.5692
Checking accuracy on validation set
Got 389 / 1000 correct (38.90)

Iteration 300, loss = 1.6112
Checking accuracy on validation set
Got 435 / 1000 correct (43.50)

Iteration 400, loss = 1.2249
Checking accuracy on validation set
Got 489 / 1000 correct (48.90)

Iteration 500, loss = 1.4368
Checking accuracy on validation set
Got 479 / 1000 correct (47.90)

Iteration 600, loss = 1.2300
Checking accuracy on validation set
Got 533 / 1000 correct (53.30)

Iteration 700, loss = 1.7592
Checking accuracy on validation set
Got 521 / 1000 correct (52.10)

Iteration 0, loss = 1.2216
Checking accuracy on validation set
Got 523 / 1000 correct (52.30)

Iteration 100, loss = 1.1924
Checking accuracy on validation set
Got 548 / 1000 correct (54.80)

Iteration 200, loss = 1.3930
Check